# ETL Partidos Atletico de Madrid

### Extraccion de los datos

In [1]:
# Importamos las librerias necesarias
from bs4 import BeautifulSoup as bs4
import requests
import pandas as pd
import warnings
warnings.filterwarnings("ignore")

In [2]:
# Creamos la url
temporada="2023-2024"
url=f"https://fbref.com/en/squads/db3b9613/{temporada}/Atletico-Madrid-Stats"

In [3]:
# Realizamos la peticion
peticion=requests.get(url)

In [4]:
# Comprobamos que el servidor esta OK
if peticion.status_code!=200:
    
    raise Exception("La pagina no esta accesible")

In [5]:
# Obtenemos el html
soup=bs4(peticion.text,"html.parser")

In [6]:
# Obtenemos la etiqueta de la tabla
tabla=soup.find(id="matchlogs_for")

In [7]:
# Obtenemos los nombres de las columnas
try:
    
    cabecera=tabla.find("thead").find("tr")

except AttributeError as e:
    
    raise Exception(f"No hay partidos de la temporada {temporada}")
    
columnas=[columna.text for columna in cabecera.find_all("th")]

In [8]:
# Obtenemos las filas de la tabla
body=tabla.find("tbody")
filas=body.find_all("tr")

In [9]:
# Funcion para extraer el contenido de las filas de la tabla
def extraerData(fila:str)->list[str]:
    
    fecha=fila.find("th").text
    
    return [fecha]+[columna.text for columna in fila.find_all("td")]

registros=list(map(extraerData, filas))

In [10]:
# Creamos un dataframe a partir de los registros
df=pd.DataFrame(registros, columns=columnas)

In [11]:
df

,Date,Time,Comp,Round,Day,Venue,Result,GF,GA,Opponent,xG,xGA,Poss,Attendance,Captain,Formation,Referee,Match Report,Notes
0,2023-08-14,21:30,La Liga,Matchweek 1,Mon,Home,W,3,1,Granada,2.4,0.9,46,"56,164",Koke,3-5-2,Juan Pulido,Match Report,
1,2023-08-20,21:30,La Liga,Matchweek 2,Sun,Away,D,0,0,Betis,0.5,0.7,47,"51,782",Jan Oblak,5-3-2,Ricardo de Burgos,Match Report,
2,2023-08-28,21:30,La Liga,Matchweek 3,Mon,Away,W,7,0,Rayo Vallecano,3.4,0.6,44,"13,966",Jan Oblak,5-3-2,José Luis Munuera,Match Report,
3,2023-09-16,16:15,La Liga,Matchweek 5,Sat,Away,L,0,3,Valencia,0.6,1.3,63,"45,363",Jan Oblak,5-3-2,Jesús Gil,Match Report,
4,2023-09-19,21:00,Champions Lg,Group stage,Tue,Away,D,1,1,it Lazio,0.9,1.2,51,"46,168",Jan Oblak,5-3-2,Slavko Vinčič,Match Report,
5,2023-09-24,21:00,La Liga,Matchweek 6,Sun,Home,W,3,1,Real Madrid,1.4,1.0,37,"69,082",Koke,5-3-2,Alberola Rojas,Match Report,
6,2023-09-28,21:30,La Liga,Matchweek 7,Thu,Away,W,2,0,Osasuna,1.5,1.0,45,"20,192",Koke,5-3-2,Juan Martínez,Match Report,
7,2023-10-01,21:00,La Liga,Matchweek 8,Sun,Home,W,3,2,Cádiz,2.7,0.8,69,"53,421",Koke,3-5-2,Javier Villanueva,Match Report,
8,2023-10-04,18:45,Champions Lg,Group stage,Wed,Home,W,3,2,nl Feyenoord,0.8,2.4,48,"61,742",Koke,3-5-2,François Letexier,Match Report,
9,2023-10-08,16:15,La Liga,Matchweek 9,Sun,Home,W,2,1,Real Sociedad,2.2,1.2,38,"65,607",Koke,3-5-2,José Luis Munuera,Match Report,


### Limpieza de los datos

In [12]:
# Seleccionamos las columnas que queremos
df=df[["Date","Time","Comp","Round","Venue","Result","GF","GA","Opponent","Poss","Attendance","Captain","Referee"]]

In [13]:
# Renombramos las columnas
df.rename({"Date":"Fecha", "Time":"Hora", "Comp":"Competicion","Round":"Ronda", "Venue":"Lugar", "Result":"Resultado",
           "GF":"Goles Marcados", "GA":"Goles Recibidos", "Opponent":"Rival", "Poss":"Posesion",
          "Attendance":"Publico", "Captain":"Capitan", "Referee":"Arbitro"}, axis=1, inplace=True)
df.head()

,Fecha,Hora,Competicion,Ronda,Lugar,Resultado,Goles Marcados,Goles Recibidos,Rival,Posesion,Publico,Capitan,Arbitro
0,2023-08-14,21:30,La Liga,Matchweek 1,Home,W,3,1,Granada,46,"56,164",Koke,Juan Pulido
1,2023-08-20,21:30,La Liga,Matchweek 2,Away,D,0,0,Betis,47,"51,782",Jan Oblak,Ricardo de Burgos
2,2023-08-28,21:30,La Liga,Matchweek 3,Away,W,7,0,Rayo Vallecano,44,"13,966",Jan Oblak,José Luis Munuera
3,2023-09-16,16:15,La Liga,Matchweek 5,Away,L,0,3,Valencia,63,"45,363",Jan Oblak,Jesús Gil
4,2023-09-19,21:00,Champions Lg,Group stage,Away,D,1,1,it Lazio,51,"46,168",Jan Oblak,Slavko Vinčič


In [14]:
# Observamos si hay partidos sin jugar (no tiene resultado)
df[df["Resultado"]==""].head()

,Fecha,Hora,Competicion,Ronda,Lugar,Resultado,Goles Marcados,Goles Recibidos,Rival,Posesion,Publico,Capitan,Arbitro
10,2023-10-21,21:00,La Liga,Matchweek 10,Away,,,,Celta Vigo,,,,
11,2023-10-25,20:00,Champions Lg,Group stage,Away,,,,sct Celtic,,,,
12,2023-10-29,21:00,La Liga,Matchweek 11,Home,,,,Alavés,,,,
13,2023-11-03,20:00,La Liga,Matchweek 12,Away,,,,Las Palmas,,,,
14,2023-11-07,21:00,Champions Lg,Group stage,Home,,,,sct Celtic,,,,


In [15]:
# Obtenemos los partidos que se han jugado
df_jugados=df[df["Resultado"]!=""]
df_jugados.head()

,Fecha,Hora,Competicion,Ronda,Lugar,Resultado,Goles Marcados,Goles Recibidos,Rival,Posesion,Publico,Capitan,Arbitro
0,2023-08-14,21:30,La Liga,Matchweek 1,Home,W,3,1,Granada,46,"56,164",Koke,Juan Pulido
1,2023-08-20,21:30,La Liga,Matchweek 2,Away,D,0,0,Betis,47,"51,782",Jan Oblak,Ricardo de Burgos
2,2023-08-28,21:30,La Liga,Matchweek 3,Away,W,7,0,Rayo Vallecano,44,"13,966",Jan Oblak,José Luis Munuera
3,2023-09-16,16:15,La Liga,Matchweek 5,Away,L,0,3,Valencia,63,"45,363",Jan Oblak,Jesús Gil
4,2023-09-19,21:00,Champions Lg,Group stage,Away,D,1,1,it Lazio,51,"46,168",Jan Oblak,Slavko Vinčič


In [16]:
# Limpiamos la competicion
competicion={"Champions Lg":"Champions League", "UEFA Cup":"Europa League", "Europa Lg":"Europa League"}
df_jugados["Competicion"]=df_jugados["Competicion"].apply(lambda x: competicion[x] if x in competicion.keys() else x)

In [17]:
# Funcion para limpiar la ronda
def limpiarRonda(ronda:str)->str:
    
    rondas={"First round":"Primera Ronda", "Second round":"Segunda Ronda", "Third round":"Tercera Ronda",
            "Play-off round":"Ronda de playoff", "Group stage":"Fase de grupos", "Round of 32":"Dieciseisavos de final",
            "Round of 16":"Octavos de final", "Quarter-finals":"Cuartos de final", "Semi-finals":"Semifinales", "Final":"Final"}
    
    if "Matchweek" in ronda:
        
        numero=ronda.split(" ")[1]
        
        return f"Jornada {numero}"
    
    elif "qualifying" in ronda:
        
        return "Fase previa"
    
    elif ronda in rondas.keys():
        
        return rondas[ronda]
    
    else:
        
        print(ronda)
        
        return ronda
    
df_jugados["Ronda"]=df_jugados["Ronda"].apply(limpiarRonda)
df_jugados.head()

,Fecha,Hora,Competicion,Ronda,Lugar,Resultado,Goles Marcados,Goles Recibidos,Rival,Posesion,Publico,Capitan,Arbitro
0,2023-08-14,21:30,La Liga,Jornada 1,Home,W,3,1,Granada,46,"56,164",Koke,Juan Pulido
1,2023-08-20,21:30,La Liga,Jornada 2,Away,D,0,0,Betis,47,"51,782",Jan Oblak,Ricardo de Burgos
2,2023-08-28,21:30,La Liga,Jornada 3,Away,W,7,0,Rayo Vallecano,44,"13,966",Jan Oblak,José Luis Munuera
3,2023-09-16,16:15,La Liga,Jornada 5,Away,L,0,3,Valencia,63,"45,363",Jan Oblak,Jesús Gil
4,2023-09-19,21:00,Champions League,Fase de grupos,Away,D,1,1,it Lazio,51,"46,168",Jan Oblak,Slavko Vinčič


In [18]:
# Limpiamos el lugar
lugares={"Home":"Local", "Away":"Visitante"}
df_jugados["Lugar"]=df_jugados["Lugar"].apply(lambda x: lugares[x] if x in lugares.keys() else x)
df_jugados.head()

,Fecha,Hora,Competicion,Ronda,Lugar,Resultado,Goles Marcados,Goles Recibidos,Rival,Posesion,Publico,Capitan,Arbitro
0,2023-08-14,21:30,La Liga,Jornada 1,Local,W,3,1,Granada,46,"56,164",Koke,Juan Pulido
1,2023-08-20,21:30,La Liga,Jornada 2,Visitante,D,0,0,Betis,47,"51,782",Jan Oblak,Ricardo de Burgos
2,2023-08-28,21:30,La Liga,Jornada 3,Visitante,W,7,0,Rayo Vallecano,44,"13,966",Jan Oblak,José Luis Munuera
3,2023-09-16,16:15,La Liga,Jornada 5,Visitante,L,0,3,Valencia,63,"45,363",Jan Oblak,Jesús Gil
4,2023-09-19,21:00,Champions League,Fase de grupos,Visitante,D,1,1,it Lazio,51,"46,168",Jan Oblak,Slavko Vinčič


In [19]:
# Limpiamos el resultado
resultados={"W":"Victoria", "D":"Empate", "L":"Derrota"}
df_jugados["Resultado"]=df_jugados["Resultado"].apply(lambda x: resultados[x])
df_jugados.head()

,Fecha,Hora,Competicion,Ronda,Lugar,Resultado,Goles Marcados,Goles Recibidos,Rival,Posesion,Publico,Capitan,Arbitro
0,2023-08-14,21:30,La Liga,Jornada 1,Local,Victoria,3,1,Granada,46,"56,164",Koke,Juan Pulido
1,2023-08-20,21:30,La Liga,Jornada 2,Visitante,Empate,0,0,Betis,47,"51,782",Jan Oblak,Ricardo de Burgos
2,2023-08-28,21:30,La Liga,Jornada 3,Visitante,Victoria,7,0,Rayo Vallecano,44,"13,966",Jan Oblak,José Luis Munuera
3,2023-09-16,16:15,La Liga,Jornada 5,Visitante,Derrota,0,3,Valencia,63,"45,363",Jan Oblak,Jesús Gil
4,2023-09-19,21:00,Champions League,Fase de grupos,Visitante,Empate,1,1,it Lazio,51,"46,168",Jan Oblak,Slavko Vinčič


In [20]:
# Creamos el marcador
df_jugados["Marcador"]=df_jugados["Goles Marcados"]+" - "+df_jugados["Goles Recibidos"]
df_jugados.head()

,Fecha,Hora,Competicion,Ronda,Lugar,Resultado,Goles Marcados,Goles Recibidos,Rival,Posesion,Publico,Capitan,Arbitro,Marcador
0,2023-08-14,21:30,La Liga,Jornada 1,Local,Victoria,3,1,Granada,46,"56,164",Koke,Juan Pulido,3 - 1
1,2023-08-20,21:30,La Liga,Jornada 2,Visitante,Empate,0,0,Betis,47,"51,782",Jan Oblak,Ricardo de Burgos,0 - 0
2,2023-08-28,21:30,La Liga,Jornada 3,Visitante,Victoria,7,0,Rayo Vallecano,44,"13,966",Jan Oblak,José Luis Munuera,7 - 0
3,2023-09-16,16:15,La Liga,Jornada 5,Visitante,Derrota,0,3,Valencia,63,"45,363",Jan Oblak,Jesús Gil,0 - 3
4,2023-09-19,21:00,Champions League,Fase de grupos,Visitante,Empate,1,1,it Lazio,51,"46,168",Jan Oblak,Slavko Vinčič,1 - 1


In [21]:
# Funcion para limpiar el equipo de la champions (eliminar abreviatura pais)
def limpiarEquipoChampions(serie:pd.Series)->pd.Series:
    
    if serie["Competicion"] in ["Champions League", "Europa League"]:
        
        equipo_dividio=serie["Rival"].split(" ")[1:]
        
        equipo=" ".join(equipo_dividio)
        
        serie["Rival"]=equipo
        
    return serie

df_jugados[["Competicion", "Rival"]]=df_jugados[["Competicion", "Rival"]].apply(limpiarEquipoChampions, axis=1)
df_jugados.head()

,Fecha,Hora,Competicion,Ronda,Lugar,Resultado,Goles Marcados,Goles Recibidos,Rival,Posesion,Publico,Capitan,Arbitro,Marcador
0,2023-08-14,21:30,La Liga,Jornada 1,Local,Victoria,3,1,Granada,46,"56,164",Koke,Juan Pulido,3 - 1
1,2023-08-20,21:30,La Liga,Jornada 2,Visitante,Empate,0,0,Betis,47,"51,782",Jan Oblak,Ricardo de Burgos,0 - 0
2,2023-08-28,21:30,La Liga,Jornada 3,Visitante,Victoria,7,0,Rayo Vallecano,44,"13,966",Jan Oblak,José Luis Munuera,7 - 0
3,2023-09-16,16:15,La Liga,Jornada 5,Visitante,Derrota,0,3,Valencia,63,"45,363",Jan Oblak,Jesús Gil,0 - 3
4,2023-09-19,21:00,Champions League,Fase de grupos,Visitante,Empate,1,1,Lazio,51,"46,168",Jan Oblak,Slavko Vinčič,1 - 1


In [22]:
# Cambiamos el tipo del dato en la columna de la posesion
df_jugados["Posesion"]=df_jugados["Posesion"].astype(int)

In [23]:
# Funcion para limpiar la cantidad de publico
def limpiarPublico(cantidad:str)->int:
    
    cantidad_str=cantidad.replace(",", "")
    
    return int(cantidad_str)

df_jugados["Publico"]=df_jugados["Publico"].apply(limpiarPublico)
df_jugados.head()

,Fecha,Hora,Competicion,Ronda,Lugar,Resultado,Goles Marcados,Goles Recibidos,Rival,Posesion,Publico,Capitan,Arbitro,Marcador
0,2023-08-14,21:30,La Liga,Jornada 1,Local,Victoria,3,1,Granada,46,56164,Koke,Juan Pulido,3 - 1
1,2023-08-20,21:30,La Liga,Jornada 2,Visitante,Empate,0,0,Betis,47,51782,Jan Oblak,Ricardo de Burgos,0 - 0
2,2023-08-28,21:30,La Liga,Jornada 3,Visitante,Victoria,7,0,Rayo Vallecano,44,13966,Jan Oblak,José Luis Munuera,7 - 0
3,2023-09-16,16:15,La Liga,Jornada 5,Visitante,Derrota,0,3,Valencia,63,45363,Jan Oblak,Jesús Gil,0 - 3
4,2023-09-19,21:00,Champions League,Fase de grupos,Visitante,Empate,1,1,Lazio,51,46168,Jan Oblak,Slavko Vinčič,1 - 1


In [24]:
# Nos quedamos con las columnas en el orden que queramos
df_final=df_jugados[["Fecha", "Hora", "Competicion", "Ronda", "Lugar", "Rival", "Marcador", "Resultado",
                    "Posesion", "Publico", "Capitan", "Arbitro"]]
df_final.head()

,Fecha,Hora,Competicion,Ronda,Lugar,Rival,Marcador,Resultado,Posesion,Publico,Capitan,Arbitro
0,2023-08-14,21:30,La Liga,Jornada 1,Local,Granada,3 - 1,Victoria,46,56164,Koke,Juan Pulido
1,2023-08-20,21:30,La Liga,Jornada 2,Visitante,Betis,0 - 0,Empate,47,51782,Jan Oblak,Ricardo de Burgos
2,2023-08-28,21:30,La Liga,Jornada 3,Visitante,Rayo Vallecano,7 - 0,Victoria,44,13966,Jan Oblak,José Luis Munuera
3,2023-09-16,16:15,La Liga,Jornada 5,Visitante,Valencia,0 - 3,Derrota,63,45363,Jan Oblak,Jesús Gil
4,2023-09-19,21:00,Champions League,Fase de grupos,Visitante,Lazio,1 - 1,Empate,51,46168,Jan Oblak,Slavko Vinčič


### Guardado de los datos

In [25]:
df_final.to_csv(f"Partidos_ATM_{temporada.replace('-','_')}.csv", index=False)